Two essential practices from the paper:First, we created a new, general set of elastic distortions that vastly expanded the size of the training set. Second, we used convolutional neural networks.

Prerequisite: If the data is scarce and if the distribution to be learned has transformation-invariance properties, generating additional data using transformations may even improve performance.

Affine distortion(仿射变换）:translate,rotation and skew..

Elastic deformation(弹性形变):

Two loss functions: Cross-entropy and MSE

We avoided using momentum, weight decay, structure-dependent learning rates, extra padding around the inputs, and averaging instead of subsampling.

Set initial weights to small random values(with std=0.05). Set lr=0.005 and multiply it by 0.3 every 100 epochs. No batch.

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)
#Sets the threshold for what messages will be logged. message level: 'FATAL','ERROR','WARN','INFO','DEBUG'
PATH = "D:\深圳\大岩\项目\intern-2018\CNN\huqy\simard"

### Architectures 1: A fully connected network with two layers.

### Architectures 2: A CNN with.
<br />--> Input(28x28x1) Preprocessing to (29x29x1)
<br />--> Convolution layer1(13x13x5 with kernel size 5, stride 2 and no padding) 
<br />--> Convolution layer2(5x5x50 with kernel szie 5, stride 2 and no padding)
<br />--> Full connected layer1(100 hidden units)
<br />--> Full connected layer2(10 output units)

In [2]:
def simard_model_fn(features, labels, mode):
    """Model function of simple cnn from Simard(2003)"""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 29, 29, 1])
    # For black-and-white photo with 29 * 29 pixels --mnist
    
    # Convolutional Layer #1 convert 29 * 29 * 1 to 13 * 13 * 5
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        strides=2,
        filters=5,
        kernel_size=[5,5],
        padding="valid",
        #kernel_initializer=tf.truncated_normal_initializer(stddev=0.005),
        activation=tf.nn.relu)
    # print(conv1.shape)
    # Convolutional Layers #2 convert 13 * 13 * 5 to 5 * 5 * 50
    conv2 = tf.layers.conv2d(
        inputs=conv1,
        strides=2,
        filters=50,
        kernel_size=[5, 5],
        padding="valid",
        #kernel_initializer=tf.truncated_normal_initializer(stddev=0.005),
        activation=tf.nn.relu)
    # print(conv2.shape)
    # Dense Layer
    conv2_flat = tf.reshape(conv2, [-1, 5 * 5 * 50])
    dense = tf.layers.dense(inputs=conv2_flat,
                            #kernel_initializer=tf.truncated_normal_initializer(stddev=0.005),
                            units=100, activation=tf.nn.relu)
    # print(dense.shape)
    # Logits Layer
    logits = tf.layers.dense(inputs=dense,
                             #kernel_initializer=tf.truncated_normal_initializer(stddev=0.005),
                             units=10)
    # print(logits.shape)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add 'softmax_tensor' to the graph. It is used for PREDICT and by the 
        # 'logging_hook'
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predicitons)
    
    # Calculate Loss using Cross-Entropy(for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    lr = 0.05
    if mode == tf.estimator.ModeKeys.TRAIN:
        step = tf.train.get_or_create_global_step()
        # test for 1 step with batchsize 100 for 600 epochs.
        boundaries = [1000, 2000, 3000, 4000, 5000]
        values = [lr, lr*0.3, lr*0.09, lr*0.027, lr*0.0081, lr*0.00243]
        lr = tf.train.piecewise_constant(step, boundaries, values)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Create the estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=simard_model_fn, model_dir=PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\深圳\\大岩\\项目\\intern-2018\\CNN\\huqy\\simard', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027CFFFC0940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Preprocessing data
1. Convert from 28\*28\*1 to 29\*29\*1

2. Affline distortion

3. Elastic deformation

In [3]:
# Load traing and eval data 
# Preprocessing the training data using affine and elastic
((train_data, train_labels),
(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required


In [4]:
# Step1: convert from 28 * 28 to 29 * 29 
def prep1(mat, size=28):
    ret = np.zeros((mat.shape[0],mat.shape[1]+1,mat.shape[2]+1))
    for i in range(mat.shape[0]):
        temp = np.vstack((np.zeros((1,size)),mat[i]))
        ret[i] = np.hstack((temp,np.zeros((size+1,1))))
    return ret
train_data = prep1(train_data)
eval_data = prep1(eval_data)

In [5]:
# Set up logging for predictions
tensor_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=50)

In [11]:
# Train the model
train_input_fn_fortest = tf.estimator.inputs.numpy_input_fn(
    x={"x":train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# Train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn_fortest,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\深圳\大岩\项目\intern-2018\CNN\huqy\simard\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2000 into D:\深圳\大岩\项目\intern-2018\CNN\huqy\simard\model.ckpt.
INFO:tensorflow:probabilities = [[0.09972279 0.10105943 0.09996591 0.10013256 0.09993591 0.09938117
  0.09993958 0.1001669  0.09972437 0.09997139]
 [0.09972278 0.10105929 0.09996612 0.10013255 0.099936   0.09938105
  0.09993965 0.10016697 0.09972438 0.09997121]
 [0.09972289 0.1010593  0.09996601 0.10013257 0.09993588 0.09938097
  0.09993966 0.10016698 0.09972438 0.09997136]
 [0.09972291 0.1010593  0.09996608 0.10013254 0.09993602 0.09938103
  0.09993959 0.10016691 0.09972432 0.09997132]
 [0.09972298 0.10105918 0.099966   0.10013276 0.09993582 0.09938113
  0.

INFO:tensorflow:loss = 2.302539348602295, step = 2001
INFO:tensorflow:Saving checkpoints for 2001 into D:\深圳\大岩\项目\intern-2018\CNN\huqy\simard\model.ckpt.
INFO:tensorflow:Loss for final step: 2.302539348602295.


### Train and evaluate

In [5]:
# Train the model with paper's methods, which is "No batch, decay learning rate, assume 1000 steps in total, decay six times" 
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [8]:
mnist_classifier.train(input_fn=train_input_fn, steps=2000)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\深圳\大岩\项目\intern-2018\CNN\huqy\simard\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4000 into D:\深圳\大岩\项目\intern-2018\CNN\huqy\simard\model.ckpt.
INFO:tensorflow:loss = 0.07243160903453827, step = 4001
INFO:tensorflow:global_step/sec: 73.3228
INFO:tensorflow:loss = 0.15583758056163788, step = 4101 (1.379 sec)
INFO:tensorflow:global_step/sec: 75.5544
INFO:tensorflow:loss = 0.0998186469078064, step = 4201 (1.308 sec)
INFO:tensorflow:global_step/sec: 76.2956
INFO:tensorflow:loss = 0.0892363041639328, step = 4301 (1.326 sec)
INFO:tensorflow:global_step/sec: 75.1727
INFO:tensorflow:loss = 0.04817202314734459, step = 4401 (1.315 sec)
INFO:tensorflow:global_step/sec: 75.4314
INFO:tensorflow:loss = 0.06703282